# Maxwell chain: response to a prescribed strain history

***

<center><i>Petr Havlásek (c) 2023, petr.havlasek@cvut.cz</i></center>

***

In [1]:
online = False

if (online):
    import micropip
    await micropip.install('ipywidgets')

import math
import numpy as np    
    
import matplotlib.pyplot as plt
    
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display


**Notation:**

>`E` ... spring stiffness [MPa]<br>

>`eta` ... dashpot viscosity [MPa day]<br>

>`tau` ... relaxation time [day] (=`eta/E`)<br>

>`t` ... time of interest [day]<br>

>`tt` ... time of loading [day]<br>

>`t1` ... time when the prescribed strain `eps1` is attained<br>

Loading history given by a piecewise-linear evolution:<br>
$\varepsilon(0) = 0$ <br>
$\varepsilon(t_1) = \varepsilon_1$ <br>
$\varepsilon(t_2) = \varepsilon_2$, $\varepsilon_2 < \varepsilon_1$<br>



## Maxwell unit class

In [2]:
class Maxwell:
    def __init__(self, E, tau, activity = False):
        self.E = E
        self.tau = tau
        self.activity = activity

    # computes relaxation function

    def R_func (self, t, tt): 
        if t >= tt:
            dt = t-tt
    
            if self.tau > 0.:
                return self.E * math.exp(-(dt)/ self.tau) 
            else:
                return self.E
                
        else:
            return 0.


## Parameters definition

In [3]:
# Reference value of spring stiffness [GPa]
E_ref = 10.
# reference value of retardation time [day]
tau_ref = 1.

# number of Maxwell units (one of which is a spring)
N = 4

# ranges for parameters of the indiviudal Maxwell units specifies powers of 10
log_E_range = 1
log_tau_range = 2

# number of time points
t_div = 100

# list which stores Maxwell units, i.e. Maxwell chain
Maxwells = []

for i in range(N):
    
    E_i = E_ref

    if i == 0:
        tau_i = 0.
    else:
        tau_i = tau_ref * 10**(i)
        
    Maxwells.append( Maxwell(E_i, tau_i) )

# spring and first full Maxwell unit are activated = Zener's model
Maxwells[0].activity = True
Maxwells[1].activity = True

# predefined settings
draw_sum = True

num_integration = False
t_div_numer = 20

eps1 = 1.e-3
t1 = tau_ref

eps2 = eps1
t2 = tau_ref * 10
    

In [4]:
## Plotting and figure updating functions

In [5]:

def update_plot():
    with output:
        output.clear_output(wait = True)
        plot_Maxwell_chain_stress()
        
        display(fig)
    
def plot_Maxwell_chain_stress():       
        
    times = np.linspace(0., t2, num = t_div )

    ax.clear()

    tt = 0.
    
    sig_tot = np.zeros(t_div)
    
    # loop over Maxwell units
    for max in Maxwells:

        # each Maxwell unit needs to be active to contribute to the relaxation
        if (max.activity):
            sig_max = []
    
            for i in range(t_div):

                t = times[i]


                # nonzero length of the loading interval
                if (t1 > 0.):
                
                    if (t <= t1):

                        if (max.tau > 0.):
                            sig_t_tt = eps1/t1 * max.E * max.tau * ( 1. - math.exp(-t/max.tau) ) 
                        else:
                            sig_t_tt = eps1/t1 * max.E * t

                    else:
                        
                        if (max.tau > 0.):
                            sig_t_tt = eps1/t1 * max.E * max.tau * (  math.exp( -(t-t1)/max.tau ) - math.exp( -t/max.tau ) )
                            sig_t_tt += (eps2-eps1)/(t2-t1) * max.E * max.tau * (  1. - math.exp( -(t-t1)/max.tau ) )
                        else:
                            sig_t_tt = ( eps1 + (eps2-eps1)/(t2-t1) * (t-t1)  ) * max.E 

                # zero length of the first loading interval
                else:

                    if (max.tau > 0.):
                        sig_t_tt = eps1 * max.E * math.exp(-t/max.tau)
                        sig_t_tt += (eps2-eps1)/(t2-t1) * max.E * max.tau * (  1. - math.exp( -(t-t1)/max.tau ) )

                    else:
                        sig_t_tt = ( eps1 + (eps2-eps1)/(t2-t1) * (t-t1)  ) * max.E 

                
                #sig_t_tt = max.R_func (times[i], tt) 
                sig_max.append(sig_t_tt)
                sig_tot[i] += sig_t_tt

            if ( Maxwells.index(max) == 0 ):
                label = "Spring"
            else:
                label = r'Maxwell$_{' + str( Maxwells.index(max) ) + "}$"  
                
            if ( Maxwells.index(max) == 0 ):
                color = "blue"
            elif ( Maxwells.index(max) == 1 ):
                color = "red"
            elif ( Maxwells.index(max) == 2 ):                
                color = "magenta"
            else:
                color = "cyan"
            
            ax.plot(times, sig_max, lw=2., color=color, label=label)


    ### Alternative approach: numerical integration, midpoint rule

    if (num_integration):

        times_numer = np.linspace(0., t2, num = t_div_numer )

        sig_tot_int = np.zeros(t_div_numer)
    
        eps = []
        for i in range(t_div_numer):
            t = times_numer[i]
    
            if (t1 == 0.):
                eps.append(eps1 + (eps2-eps1)/(t2-t1) * (t-t1) )
    
            else:
                if (t <= t1):
                    eps.append(eps1/t1 * t)
                else:
                    eps.append(eps1 + (eps2-eps1)/(t2-t1) * (t-t1) )
       
        # loop over Maxwell units
        for max in Maxwells:
    
            # each Maxwell unit needs to be active to contribute to the relaxation
            if (max.activity): 
    
                sig_max_int = []
                
                for i in range(t_div_numer):
    
                    d_sigma = 0.
    
                    if (t1 == 0.):
                        d_sigma += eps1 * max.R_func (times_numer[i], 0.)
                    
                    if (i > 0):
                        for j in range(i):
                            
                            tt = (times_numer[j] + times_numer[j+1])/2.
                            d_eps = eps[j+1] - eps[j]
                            d_sigma += d_eps * max.R_func (times_numer[i], tt)
    
                    sig_max_int.append(d_sigma)
    
                    sig_tot_int[i] += d_sigma
            
    
                if ( Maxwells.index(max) == 0 ):
                    label = "Spring"
                else:
                    label = r'Maxwell$_{' + str( Maxwells.index(max) ) + "}$"  
                    
                if ( Maxwells.index(max) == 0 ):
                    color = "blue"
                elif ( Maxwells.index(max) == 1 ):
                    color = "red"
                elif ( Maxwells.index(max) == 2 ):                
                    color = "magenta"
                else:
                    color = "cyan"
                
                ax.scatter(times_numer, sig_max_int, marker = 'x', color=color, s = 30, linewidths = 1.)

     

    # plot vertical line corresponding to t1
    ax.axvline(x = t1, lw=4., color="green", linestyle = "-")

    # draws the response (or relaxation function) of the entire chain
    if (draw_sum):
        ax.plot(times, sig_tot, color="black", lw = 2., label=r"$\sigma_{tot}$")

        if (num_integration):
            ax.scatter(times_numer, sig_tot_int, color="black", marker = 'x', linewidths = 1., s = 30)

        

    ax.set_xlim([0., t2])
    
    ax.legend()
    ax.set_xlabel('Time, $t$ [day]', fontsize=14)
    ax.set_ylabel('Stress, $\sigma$ [MPa]', fontsize=14)

    ax.grid(True)


## GUI

In [6]:
step = 1./20.

## sliders
# degenerated Maxwell unit (spring)

i = 0

E = Maxwells[i].E

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range

E_0_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E0 [GPa]', continuous_update=False, orientation='vertical')
Max_0_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')


###
# full Maxwell units
###

i = 1

E = Maxwells[i].E
tau = Maxwells[i].tau

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range
log_tau_min = math.floor(math.log10(tau)) - log_tau_range
log_tau_max = math.ceil(math.log10(tau)) + log_tau_range

E_1_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E1 [GPa]', continuous_update=False, orientation='vertical')
tau_1_slide = widgets.FloatLogSlider(min=log_tau_min, max=log_tau_max, value=tau, step=step, description='tau1 [day]', continuous_update=False, orientation='vertical')
Max_1_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')

###

i = 2

E = Maxwells[i].E 
tau = Maxwells[i].tau

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range
log_tau_min = math.floor(math.log10(tau)) - log_tau_range
log_tau_max = math.ceil(math.log10(tau)) + log_tau_range

E_2_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E2 [GPa]', continuous_update=False, orientation='vertical')
tau_2_slide = widgets.FloatLogSlider(min=log_tau_min, max=log_tau_max, value=tau, step=step, description='tau2 [day]', continuous_update=False, orientation='vertical')
Max_2_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')

###

i = 3

E = Maxwells[i].E
tau = Maxwells[i].tau

log_E_min = math.floor(math.log10(E)) - log_E_range
log_E_max = math.ceil(math.log10(E)) + log_E_range
log_tau_min = math.floor(math.log10(tau)) - log_tau_range
log_tau_max = math.ceil(math.log10(tau)) + log_tau_range

E_3_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=E, step=step, description='E3 [GPa]', continuous_update=False, orientation='vertical')
tau_3_slide = widgets.FloatLogSlider(min=log_tau_min, max=log_tau_max, value=tau, step=step, description='tau3 [day]', continuous_update=False, orientation='vertical')
Max_3_activity = widgets.Checkbox(value=Maxwells[i].activity, description='active')

###

draw_sum_checkbox = widgets.Checkbox(value=draw_sum, description='draw sum')

###
# time t1 which defines the loading rate

log_t1_min = math.floor(math.log10(tau_ref/100. )) 
log_t1_max = math.ceil(math.log10(tau_ref*10.))
t1_slide = widgets.FloatSlider(min=0., max=t2, value=t1, step=step, description='t1 [day]', continuous_update=False, orientation='horizontal')

eps1_slide = widgets.FloatLogSlider(min=math.log10(eps1)-1, max=math.log10(eps1)+1, value=math.log10(eps1), step=step, description='eps1 [-]', continuous_update=False, orientation='horizontal')

eps2_to_eps1_slide = widgets.FloatSlider(min=0., max=1., value=eps2/eps1, step=step, description='eps2/eps1 [-]', continuous_update=False, orientation='horizontal')

num_integration_checkbox = widgets.Checkbox(value=num_integration, description='num. integration')
t_div_numer_slide = widgets.IntSlider(min=4., max=50., value=t_div_numer, step=2, description='# points [-]', continuous_update=False, orientation='horizontal')

###




def on_E_0_change(change):
    Maxwells[0].E = change['new']
    update_plot()

def on_activity_0_change(change):
    Maxwells[0].activity = change['new']
    update_plot()

def on_E_1_change(change):
    Maxwells[1].E = change['new']
    update_plot()

def on_tau_1_change(change):
    Maxwells[1].tau = change['new']
    update_plot()    

def on_activity_1_change(change):
    Maxwells[1].activity = change['new']
    update_plot()    

def on_E_2_change(change):
    Maxwells[2].E = change['new']
    update_plot()
         

def on_tau_2_change(change):
    Maxwells[2].tau = change['new']
    update_plot()   

def on_activity_2_change(change):
    Maxwells[2].activity = change['new']
    update_plot()    
    
def on_E_3_change(change):
    Maxwells[3].E = change['new']
    update_plot()
         
def on_tau_3_change(change):
    Maxwells[3].tau = change['new']
    update_plot()   

def on_activity_3_change(change):
    Maxwells[3].activity = change['new']
    update_plot()    
    
def on_draw_sum_change(change):

    global draw_sum
    draw_sum = change['new']
    update_plot()    

def on_t1_change(change):
    global t1
    t1 = change['new']
    update_plot()   

def on_eps1_change(change):
    global eps1
    eps1 = change['new']

    global eps2
    eps2 = eps2_to_eps1_slide.value * eps1
    
    update_plot()   

def on_eps2_to_eps1_change(change):
    global eps2
    eps2 = change['new'] * eps1
    update_plot()   
    
def on_num_integration_change(change):
    global num_integration
    num_integration = change['new']
    update_plot()    

def on_t_div_numer_change(change):
    global t_div_numer 
    t_div_numer = change['new']
    update_plot()    

    

Max_0_activity.observe(on_activity_0_change, names = 'value')
E_0_slide.observe(on_E_0_change, names = 'value')

Max_1_activity.observe(on_activity_1_change, names = 'value')
E_1_slide.observe(on_E_1_change, names = 'value')
tau_1_slide.observe(on_tau_1_change, names = 'value')

Max_2_activity.observe(on_activity_2_change, names = 'value')
E_2_slide.observe(on_E_2_change, names = 'value')
tau_2_slide.observe(on_tau_2_change, names = 'value')

Max_3_activity.observe(on_activity_3_change, names = 'value')
E_3_slide.observe(on_E_3_change, names = 'value')
tau_3_slide.observe(on_tau_3_change, names = 'value')



draw_sum_checkbox.observe(on_draw_sum_change, names = 'value')

t1_slide.observe(on_t1_change, names = 'value')
eps1_slide.observe(on_eps1_change, names = 'value')
eps2_to_eps1_slide.observe(on_eps2_to_eps1_change, names = 'value')


num_integration_checkbox.observe(on_num_integration_change, names = 'value')
t_div_numer_slide.observe(on_t_div_numer_change, names = 'value')



par_0 = widgets.VBox([Max_0_activity, E_0_slide])

par_1 = widgets.VBox([Max_1_activity, widgets.HBox([E_1_slide, tau_1_slide])])

par_2 = widgets.VBox([Max_2_activity, widgets.HBox([E_2_slide, tau_2_slide])])

par_3 = widgets.VBox([Max_3_activity, widgets.HBox([E_3_slide, tau_3_slide])])


pars = widgets.HBox([par_0, par_1, par_2, par_3])



## Updating 

In [7]:

output = widgets.Output()

fig, ax = plt.subplots(1, 1, figsize=(10,5))

plt.rcParams.update({'font.size': 14})
plt.close(fig)

update_plot()

display( widgets.HBox( [ widgets.VBox([t1_slide, eps1_slide, eps2_to_eps1_slide]), 
                        widgets.VBox([draw_sum_checkbox, num_integration_checkbox, t_div_numer_slide] ) ] ), output, pars) 


Output()